# 파일 불러오기
- 날짜 없는 원본 파일 사용

In [1]:
import pandas as pd

# Load the CSV file
file_path = "./Final Results Extended.csv"
df = pd.read_csv(file_path)
df.head()

,Time_Now,Blanking_Util,Blanking_SKU1_Queue,Blanking_SKU2_Queue,Blanking_SKU3_Queue,Blanking_SKU4_Queue,Press1_Util,Press2_Util,Press3_Util,Press4_Util,...,SKU3_NVA_Time,SKU3_Transport_Time,SKU3_Wait_Time,SKU3_Other_Time,SKU4_VA_Time,SKU4_NVA_Time,SKU4_Transport_Time,SKU4_Wait_Time,SKU4_Other_Time,Blanking_Queue
0,24,0.846367,0.045715,0.056373,0.055737,0.035849,0.410297,0.434561,0.481388,0.399992,...,0.0,0.537617,0.453650,0.0,1.523338,0.0,0.536243,0.473453,0.0,58.361452
1,24,0.851097,0.051937,0.052934,0.038512,0.042248,0.455471,0.454445,0.387975,0.442986,...,0.0,0.536764,0.473677,0.0,1.523344,0.0,0.534992,0.464380,0.0,62.830599
2,24,0.846115,0.052210,0.047499,0.043181,0.040979,0.496717,0.450816,0.417308,0.352829,...,0.0,0.535925,0.424090,0.0,1.523403,0.0,0.535077,0.475330,0.0,59.365867
3,24,0.841306,0.051769,0.035436,0.046788,0.052362,0.433749,0.363004,0.443909,0.456036,...,0.0,0.535232,0.430992,0.0,1.523381,0.0,0.533233,0.463801,0.0,56.698528
4,24,0.859599,0.045874,0.046802,0.044507,0.057210,0.418329,0.396826,0.499273,0.472454,...,0.0,0.538142,0.502614,0.0,1.523363,0.0,0.537223,0.449320,0.0,65.784631


# 데이터 전처리

## 상수컬럼 및 전부 0인 컬럼 제거
- Time_Now에 동일한 24 값 -> 제거
- 결과적으로 22개 컬럼 제거
- Queue 변수 중 제거된 거 : 'Paint1_Queue', 'Paint2_Queue', 'Cell1_Queue', 'Cell2_Queue', 'Cell3_Queue', 'Cell4_Queue'
- Cell별 SKU 생산량 변수 중 제거된 거 : 'c_Cell1_SKU3', 'c_Cell2_SKU1', 'c_Cell2_SKU3', 'c_Cell3_SKU1', 'c_Cell3_SKU4', 'c_Cell4_SKU1', 'c_Cell4_SKU2'

In [2]:
# =========================
# 1. 상수컬럼 및 전부 0인 컬럼 제거
# =========================

# (1) 상수컬럼: 데이터가 모두 같은 값인 컬럼
const_cols = [col for col in df.columns if df[col].nunique() == 1]

# (2) 전부 0인 컬럼
zero_cols = [col for col in df.columns if (df[col] == 0).all()]

# 합치기 (중복 제거)
drop_cols = list(set(const_cols + zero_cols))

print(f"🗑 제거 대상 컬럼 수: {len(drop_cols)}개")
print(drop_cols)

# 실제 제거
df = df.drop(columns=drop_cols)

print(f"✅ 제거 후 컬럼 수: {df.shape[1]}")

🗑 제거 대상 컬럼 수: 22개
['SKU2_NVA_Time', 'Cell1_Queue', 'c_Cell2__SKU1', 'SKU3_Other_Time', 'SKU3_NVA_Time', 'c_Cell4__SKU1', 'c_Cell1__SKU3', 'Time_Now', 'Paint2_Queue', 'SKU4_NVA_Time', 'Cell2_Queue', 'c_Cell4__SKU2', 'Cell4_Queue', 'SKU1_Other_Time', 'c_Cell3__SKU4', 'Paint1_Queue', 'Cell3_Queue', 'c_Cell2__SKU3', 'c_Cell3__SKU1', 'SKU1_NVA_Time', 'SKU4_Other_Time', 'SKU2_Other_Time']
✅ 제거 후 컬럼 수: 56


## 결측치 확인 -> 1~3개씩 존재 => 해당 행 drop (행 3개 제거)

In [3]:
# =========================
# 2. 결측치 확인
# =========================
missing_summary = df.isnull().sum()
missing_summary = missing_summary[missing_summary > 0]

if missing_summary.empty:
    print("✅ 결측치 없음")
else:
    print("⚠️ 결측치 요약:")
    print(missing_summary)

⚠️ 결측치 요약:
Blanking_SKU4_Queue        1
Press1_Util                1
Press2_Util                1
Press3_Util                1
Press4_Util                1
Press1_Queue               1
Press2_Queue               1
Press3_Queue               1
Press4_Queue               1
Cell1_Util                 1
Cell2_Util                 1
Cell3_Util                 1
Cell4_Util                 1
Warehouse1_Queue           1
Warehouse_2_Queue          1
Warehouse_3_Queue          1
Warehouse_4_Queue          1
c_Cycle1                   1
c_Cycle2                   1
c_Cycle3                   1
c_Cycle4                   1
c_Cell1_SKU1               1
c_Cell1__SKU2              1
c_Cell1__SKU4              1
c_Cell2__SKU2              1
c_Cell2__SKU4              1
c_Cell3__SKU2              1
c_Cell3__SKU3              1
c_Cell4__SKU3              1
c_Cell4__SKU4              1
Paint1_Util                1
Paint2_Util                1
Quality_Util               1
Quality_Queue              1
For

In [4]:
# =========================
# 3. 결측치 제거
# =========================
if not missing_summary.empty:
    before = df.shape[0]
    df = df.dropna()  # 결측치가 하나라도 있는 행 제거
    after = df.shape[0]
    print(f"✅ 결측치가 있는 행 {before - after}개 제거 완료 (남은 행: {after})")

print()
missing_summary2 = df.isnull().sum()
missing_summary2 = missing_summary2[missing_summary2 > 0]
print(missing_summary2)

✅ 결측치가 있는 행 3개 제거 완료 (남은 행: 132673)

Series([], dtype: int64)


# Warehouse Queue -> 네이밍을 Cell Queue 로 변경 (with 넘버링 매핑)
- 이름 변경 매핑: {'Warehouse1_Queue': 'Cell1_Queue', 'Warehouse_2_Queue': 'Cell2_Queue', 'Warehouse_3_Queue': 'Cell3_Queue', 'Warehouse_4_Queue': 'Cell4_Queue'}

In [5]:
# =========================
# 1) Warehouse Queue 컬럼 찾기
# =========================
warehouse_cols = [c for c in df.columns if "warehouse" in c.lower() and "queue" in c.lower()]
print("Warehouse Queue cols:", warehouse_cols)

# =========================
# 2) Warehouse 번호 → Cell 번호로 변환
# =========================
rename_map = {}
for w in warehouse_cols:
    # 숫자 추출 (예: Warehouse1_Queue → 1)
    num = ''.join(filter(str.isdigit, w))
    if num:
        new_name = f"Cell{num}_Queue"
        rename_map[w] = new_name

# =========================
# 3) 컬럼명 변경
# =========================
df = df.rename(columns=rename_map)

print("✅ 이름 변경 매핑:", rename_map)
print("변경 후 Queue 관련 컬럼:", [c for c in df.columns if "queue" in c.lower()])

Warehouse Queue cols: ['Warehouse1_Queue', 'Warehouse_2_Queue', 'Warehouse_3_Queue', 'Warehouse_4_Queue']
✅ 이름 변경 매핑: {'Warehouse1_Queue': 'Cell1_Queue', 'Warehouse_2_Queue': 'Cell2_Queue', 'Warehouse_3_Queue': 'Cell3_Queue', 'Warehouse_4_Queue': 'Cell4_Queue'}
변경 후 Queue 관련 컬럼: ['Blanking_SKU1_Queue', 'Blanking_SKU2_Queue', 'Blanking_SKU3_Queue', 'Blanking_SKU4_Queue', 'Press1_Queue', 'Press2_Queue', 'Press3_Queue', 'Press4_Queue', 'Cell1_Queue', 'Cell2_Queue', 'Cell3_Queue', 'Cell4_Queue', 'Quality_Queue', 'Forklift_Blanking_Queue', 'Forklift_Press_Queue', 'Forklift_Assembly_Queue', 'Blanking_Queue']


# Target 컬럼 생성

## 전체 기준 병목 파악 -> 새 컬럼에 추가 (Bottleneck_actual , Bottleneck_actual_val)

In [6]:
# =========================
# 1) Queue 컬럼 찾기 (대소문자 무시)
# =========================
queue_cols = [col for col in df.columns if "queue" in col.lower()]

print(f"Queue 관련 컬럼 수: {len(queue_cols)}")
print(queue_cols[:10])  # 앞 10개만 확인

# =========================
# 2) 병목 공정 컬럼 생성
# =========================
# 각 행에서 queue 값이 가장 큰 컬럼명
df["Bottleneck_actual"] = df[queue_cols].idxmax(axis=1)

# 해당 queue의 실제 최대값도 같이 기록하고 싶다면
df["Bottleneck_val"] = df[queue_cols].max(axis=1)

# =========================
# 3) 결과 확인
# =========================
print(df[["Bottleneck_actual", "Bottleneck_val"]].tail())

Queue 관련 컬럼 수: 17
['Blanking_SKU1_Queue', 'Blanking_SKU2_Queue', 'Blanking_SKU3_Queue', 'Blanking_SKU4_Queue', 'Press1_Queue', 'Press2_Queue', 'Press3_Queue', 'Press4_Queue', 'Cell1_Queue', 'Cell2_Queue']
              Bottleneck_actual  Bottleneck_val
132670  Forklift_Blanking_Queue      153.012984
132671              Cell1_Queue      185.031493
132672              Cell1_Queue      302.349198
132673              Cell1_Queue      295.667368
132674              Cell1_Queue      166.628981


In [7]:
# Bottleneck 컬럼의 빈도 계산
bottleneck_counts = df["Bottleneck_actual"].value_counts().head(10)

bottleneck_counts

Bottleneck_actual
Cell1_Queue                77629
Forklift_Blanking_Queue    53681
Cell3_Queue                 1307
Cell4_Queue                   36
Forklift_Press_Queue           9
Press4_Queue                   6
Press2_Queue                   5
Name: count, dtype: int64

## 주요 공정 단계(블랭킹, 프레스, 조립셀) 기준 병목 파악 -> 새 컬럼에 추가
- Bottleneck_actual_Blanking , Bottleneck_actual_Press , Bottleneck_actual_Cell
- Bottleneck_val_Blanking , Bottleneck_val_Press , Bottleneck_val_Cell

In [8]:
# ================================
# 11. 공정별 실제 병목 레이블 & 값 생성
# ================================
stage_groups = {
    "Blanking": ["Blanking_SKU1_Queue", "Blanking_SKU2_Queue",
                 "Blanking_SKU3_Queue", "Blanking_SKU4_Queue"],
    "Press":    ["Press1_Queue", "Press2_Queue", "Press3_Queue", "Press4_Queue"],
    "Cell":     ["Cell1_Queue", "Cell2_Queue", "Cell3_Queue", "Cell4_Queue"]
}

for stage, cols in stage_groups.items():
    # 병목 컬럼명 (어느 라인/설비가 병목인지)
    df[f"Bottleneck_actual_{stage}"] = df[cols].idxmax(axis=1)
    # 병목 값 (queue 크기)
    df[f"Bottleneck_val_{stage}"] = df[cols].max(axis=1)

# ================================
# 12. 결과 확인
# ================================
check_cols = [c for c in df.columns if c.startswith("Bottleneck_")]
print(df[check_cols].head())

         Bottleneck_actual  Bottleneck_val Bottleneck_actual_Blanking  \
0  Forklift_Blanking_Queue      157.256741        Blanking_SKU2_Queue   
1  Forklift_Blanking_Queue      166.488725        Blanking_SKU2_Queue   
2              Cell1_Queue      335.401556        Blanking_SKU1_Queue   
3  Forklift_Blanking_Queue      147.448407        Blanking_SKU4_Queue   
4  Forklift_Blanking_Queue      150.397600        Blanking_SKU4_Queue   

   Bottleneck_val_Blanking Bottleneck_actual_Press  Bottleneck_val_Press  \
0                 0.056373            Press2_Queue             65.273786   
1                 0.052934            Press1_Queue             74.772823   
2                 0.052210            Press2_Queue             72.901066   
3                 0.052362            Press1_Queue             65.091192   
4                 0.057210            Press1_Queue             65.075312   

  Bottleneck_actual_Cell  Bottleneck_val_Cell  
0            Cell1_Queue            98.155568  
1       

## 제품군(SKU) 기준 병목 파악 -> 새 컬럼에 추가
- Bottleneck_actual_SKU1 , Bottleneck_actual_SKU2 , Bottleneck_actual_SKU3 , Bottleneck_actual_SKU4
- Bottleneck_val_SKU1 , Bottleneck_val_SKU2 , Bottleneck_val_SKU3, Bottleneck_val_SKU4

### 컬럼명 맞추기

In [9]:
# =========================
# 컬럼명 정규화: 언더스코어 여러 개 → 1개로 통일
# =========================
df.columns = df.columns.str.replace(r"__+", "_", regex=True)

# 확인
for sku in ["SKU1","SKU2","SKU3","SKU4"]:
    related_cols = [c for c in df.columns if f"Cell" in c and sku in c]
    print(sku, related_cols)

SKU1 ['c_Cell1_SKU1']
SKU2 ['c_Cell1_SKU2', 'c_Cell2_SKU2', 'c_Cell3_SKU2']
SKU3 ['c_Cell3_SKU3', 'c_Cell4_SKU3']
SKU4 ['c_Cell1_SKU4', 'c_Cell2_SKU4', 'c_Cell4_SKU4']


### 조립셀 Queue 값 -> SKU별로 분배
- 각 Cell Queue 전체량을 SKU별 담당 비율에 따라 나눠서, SKU별 Cell Queue를 생성 -> SKU별로 여러 Cell에서 받은 몫을 합쳐 최종 SKU Cell Queue를 계산
- 각 행에 대해 동적으로 SKU별 분배 수행

In [10]:
for sku in ["SKU1","SKU2","SKU3","SKU4"]:
    related_cols = [c for c in df.columns if f"_{sku}" in c and "c_Cell" in c]
    print(sku, related_cols)

SKU1 ['c_Cell1_SKU1']
SKU2 ['c_Cell1_SKU2', 'c_Cell2_SKU2', 'c_Cell3_SKU2']
SKU3 ['c_Cell3_SKU3', 'c_Cell4_SKU3']
SKU4 ['c_Cell1_SKU4', 'c_Cell2_SKU4', 'c_Cell4_SKU4']


In [11]:
# ================================
# 2. SKU 경로 정의
# ================================
sku_paths = {
    "SKU1": {"blanking": "Blanking_SKU1_Queue","press": "Press1_Queue"},
    "SKU2": {"blanking": "Blanking_SKU2_Queue","press": "Press2_Queue"},
    "SKU3": {"blanking": "Blanking_SKU3_Queue","press": "Press3_Queue"},
    "SKU4": {"blanking": "Blanking_SKU4_Queue","press": "Press4_Queue"},
}

# ================================
# 3. Cell → SKU 비율 기반 분배
# ================================
sku_cell_vals = {sku: pd.Series(0, index=df.index) for sku in sku_paths}

for cell_num in range(1, 5):  # Cell1 ~ Cell4
    cell_col = f"Cell{cell_num}_Queue"
    
    if cell_col not in df.columns:
        continue
    
    # 해당 Cell Queue 값
    cell_vals = df[cell_col]
    
    for sku in sku_paths.keys():
        ratio_col = f"c_Cell{cell_num}_{sku}"
        if ratio_col in df.columns:
            # SKU별 Cell Queue = Cell Queue × 비율
            sku_cell_vals[sku] += cell_vals * df[ratio_col]

# ================================
# 4. 결과 DataFrame 구성 (컬럼명 변경)
# ================================
sku_cell_df = pd.DataFrame({
    f"Cell_{sku}_Queue": vals for sku, vals in sku_cell_vals.items()
})

print("=== SKU별로 분배된 Cell Queue (앞부분 10행) ===")
print(sku_cell_df.head(10))

# 원래 df에 붙이고 싶다면:
df = pd.concat([df, sku_cell_df], axis=1)

=== SKU별로 분배된 Cell Queue (앞부분 10행) ===
   Cell_SKU1_Queue  Cell_SKU2_Queue  Cell_SKU3_Queue  Cell_SKU4_Queue
0     1.370939e+06     1.192535e+06     1.247366e+06     8.856453e+05
1     2.451033e+06     1.958747e+06     9.126624e+05     1.182048e+06
2     5.568672e+06     3.817909e+06     9.283182e+05     1.242360e+06
3     1.647598e+06     1.114051e+06     1.090078e+06     1.048065e+06
4     1.553549e+06     1.232362e+06     1.476219e+06     1.102118e+06
5     2.101007e+06     1.931448e+06     1.167199e+06     1.028449e+06
6     1.851074e+06     1.886183e+06     6.287604e+05     8.966812e+05
7     2.972045e+06     2.248654e+06     1.170633e+06     1.487739e+06
8     5.720726e+06     4.591313e+06     9.949638e+05     1.545606e+06
9     5.471222e+06     3.388148e+06     7.908985e+05     1.160693e+06


In [12]:
# ================================
# 7. SKU별 병목 공정 예측
# ================================
sku_paths = {
    "SKU1": {"blanking": "Blanking_SKU1_Queue","press": "Press1_Queue", "cell": "Cell_SKU1_Queue"},
    "SKU2": {"blanking": "Blanking_SKU2_Queue","press": "Press2_Queue", "cell": "Cell_SKU2_Queue"},
    "SKU3": {"blanking": "Blanking_SKU3_Queue","press": "Press3_Queue", "cell": "Cell_SKU3_Queue"},
    "SKU4": {"blanking": "Blanking_SKU4_Queue","press": "Press4_Queue", "cell": "Cell_SKU4_Queue"},
}

for sku, paths in sku_paths.items():
    # 각 SKU별 병목 후보 Stage 값 모으기
    stage_df = df[[paths["blanking"], paths["press"], paths["cell"]]].copy()
    stage_df.columns = ["Blanking", "Press", "Cell"]  # 축 이름 단순화
    
    # 각 시점별 최대값 컬럼명 → 병목 공정
    df[f"Bottleneck_actual_{sku}"] = stage_df.idxmax(axis=1)

    # 각 시점별 병목 수치 (최대값)
    df[f"Bottleneck_val_{sku}"] = stage_df.max(axis=1)

# ================================
# 8. 결과 확인
# ================================
print(df[[c for c in df.columns if c.startswith("Bottleneck_actual_")]].head())
print(df[[c for c in df.columns if c.startswith("Bottleneck_val_")]].head())

  Bottleneck_actual_Blanking Bottleneck_actual_Press Bottleneck_actual_Cell  \
0        Blanking_SKU2_Queue            Press2_Queue            Cell1_Queue   
1        Blanking_SKU2_Queue            Press1_Queue            Cell1_Queue   
2        Blanking_SKU1_Queue            Press2_Queue            Cell1_Queue   
3        Blanking_SKU4_Queue            Press1_Queue            Cell1_Queue   
4        Blanking_SKU4_Queue            Press1_Queue            Cell1_Queue   

  Bottleneck_actual_SKU1 Bottleneck_actual_SKU2 Bottleneck_actual_SKU3  \
0                   Cell                   Cell                   Cell   
1                   Cell                   Cell                   Cell   
2                   Cell                   Cell                   Cell   
3                   Cell                   Cell                   Cell   
4                   Cell                   Cell                   Cell   

  Bottleneck_actual_SKU4  
0                   Cell  
1                   Cell  

# 랜덤 분할 - Train 70 : Val 15 : Test 15

In [13]:
# from sklearn.model_selection import train_test_split

# train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42, shuffle=True)
# train_df, val_df = train_test_split(train_val_df, test_size=0.1765, random_state=42, shuffle=True)
# # 0.1765 ≈ 15% / 85%

# print(f"Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}")

In [14]:
# # 저장 경로 지정 (원하는 폴더로 바꾸세요)
# train_path = "./bottleneck_train.csv"
# val_path   = "./bottleneck_val.csv"
# test_path  = "./bottleneck_test.csv"

# # CSV 저장 (인덱스 제외)
# train_df.to_csv(train_path, index=False, encoding="utf-8-sig")
# val_df.to_csv(val_path, index=False, encoding="utf-8-sig")
# test_df.to_csv(test_path, index=False, encoding="utf-8-sig")

# print("✅ CSV 저장 완료")
# print(f"Train → {train_path} ({train_df.shape})")
# print(f"Val   → {val_path} ({val_df.shape})")
# print(f"Test  → {test_path} ({test_df.shape})")

In [15]:
import pandas as pd

# 경로 지정
train_path = "./bottleneck_train.csv"
val_path   = "./bottleneck_val.csv"
test_path  = "./bottleneck_test.csv"

# 불러오기
train_df = pd.read_csv(train_path)
val_df   = pd.read_csv(val_path)
test_df  = pd.read_csv(test_path)

# 하나로 합치고 싶으면
df = pd.concat([train_df, val_df, test_df], axis=0).reset_index(drop=True)

print("✅ 불러오기 완료")
print("Train:", train_df.shape, "Val:", val_df.shape, "Test:", test_df.shape)
print("전체 df:", df.shape)

✅ 불러오기 완료
Train: (92867, 76) Val: (19905, 76) Test: (19901, 76)
전체 df: (132673, 76)


# 독립변수 분리
- 전체 기준 병목 예측
    - 분류 : Bottleneck_actual
    - 회귀 : Bottleneck_actual_val 
- 공정 단계 기준 병목 예측
    - 분류 : Bottleneck_actual_Blanking , Bottleneck_actual_Press , Bottleneck_actual_Cell
    - 회귀 : Bottleneck_val_Blanking , Bottleneck_val_Press , Bottleneck_val_Cell 
- 제품군(SKU) 기준 병목 예측
    - 분류 : Bottleneck_actual_SKU1 , Bottleneck_actual_SKU2 , Bottleneck_actual_SKU3 , Bottleneck_actual_SKU4
    - 회귀 : Bottleneck_val_SKU1 , Bottleneck_val_SKU2 , Bottleneck_val_SKU3, Bottleneck_val_SKU4

In [16]:
# 타깃 후보 컬럼 전부 정리
target_cols = [
    # 전체 기준
    "Bottleneck_actual", "Bottleneck_val",
    # 공정 단계 기준
    "Bottleneck_actual_Blanking", "Bottleneck_actual_Press", "Bottleneck_actual_Cell",
    "Bottleneck_val_Blanking", "Bottleneck_val_Press", "Bottleneck_val_Cell",
    # SKU 기준
    "Bottleneck_actual_SKU1", "Bottleneck_actual_SKU2", "Bottleneck_actual_SKU3", "Bottleneck_actual_SKU4",
    "Bottleneck_val_SKU1", "Bottleneck_val_SKU2", "Bottleneck_val_SKU3", "Bottleneck_val_SKU4"
]

# 독립변수(X)는 이 타깃 후보들을 제외한 것들만
X_train = train_df.drop(columns=[c for c in target_cols if c in train_df.columns])
X_val   = val_df.drop(columns=[c for c in target_cols if c in val_df.columns])
X_test  = test_df.drop(columns=[c for c in target_cols if c in test_df.columns])

print("✅ 독립변수 컬럼 수:", X_train.shape[1])
print("예시 컬럼:", X_train.columns[:10].tolist())

✅ 독립변수 컬럼 수: 60
예시 컬럼: ['Blanking_Util', 'Blanking_SKU1_Queue', 'Blanking_SKU2_Queue', 'Blanking_SKU3_Queue', 'Blanking_SKU4_Queue', 'Press1_Util', 'Press2_Util', 'Press3_Util', 'Press4_Util', 'Press1_Queue']


# 전체 기준 병목 예측 모델링
- 분류 : Bottleneck_actual
- 회귀 : Bottleneck_actual_val 

### 추가 처리
- 트리 기반 모델(RandomForest, XGB) → 그대로 둠 (스케일링 불필요)
- Logistic / Ridge / MLP → StandardScaler 적용
- MLP → PyTorch 기반 GPU 학습

In [19]:
# import numpy as np
# import pandas as pd
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score
# import joblib

# # =========================
# # 1) 타깃 인코딩 (여기서 le 먼저 생성!)
# # =========================
# le = LabelEncoder()  # ✅ 추가

# y_train_cls = le.fit_transform(train_df["Bottleneck_actual"])
# y_val_cls   = le.transform(val_df["Bottleneck_actual"])
# y_test_cls  = le.transform(test_df["Bottleneck_actual"])

# # =========================
# # 2) Logistic(best) 학습/평가
# # =========================
# log_best = make_pipeline(
#     StandardScaler(),
#     LogisticRegression(
#         C=0.1,
#         penalty="l1",
#         class_weight="balanced",
#         solver="saga",
#         max_iter=5000,
#         n_jobs=-1,
#         random_state=42,
#     )
# )

# # train+val로 재학습
# X_tr_full = pd.concat([X_train, X_val], axis=0, ignore_index=True)  # ✅ pd.concat 권장
# y_tr_full = np.concatenate([y_train_cls, y_val_cls])

# log_best.fit(X_tr_full, y_tr_full)

# # 테스트
# y_pred = log_best.predict(X_test)
# acc = accuracy_score(y_test_cls, y_pred)
# f1 = f1_score(y_test_cls, y_pred, average="macro")
# bal_acc = balanced_accuracy_score(y_test_cls, y_pred)
# print(f"Logistic(best) | Acc={acc:.4f}, F1={f1:.4f}, BalAcc={bal_acc:.4f}")

# # =========================
# # 3) 결과/모델 저장 (.joblib + .pkl)
# # =========================
# pd.DataFrame([{
#     "name": "Logistic(best)",
#     "Acc": acc, "F1": f1, "BalAcc": bal_acc
# }]).to_csv("classification_logistic_best_only.csv", index=False, encoding="utf-8-sig")

# package = {"model": log_best, "label_encoder": le}

# joblib.dump(package, "logistic_best_model.joblib")
# print("💾 Saved: logistic_best_model.joblib")

# joblib.dump(package, "logistic_best_model.pkl")
# print("💾 Saved: logistic_best_model.pkl")

Logistic(best) | Acc=0.9964, F1=0.8949, BalAcc=0.9433
💾 Saved: logistic_best_model.joblib
💾 Saved: logistic_best_model.pkl


C:\Users\Administrator\PycharmProjects\PythonProject\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [20]:
# # =========================
# # 4) 테스트셋 예측 결과 + 확률 저장
# # =========================

# # y_test_cls 는 정수 인코딩 값 → 원래 라벨로 복원
# y_true_labels = le.inverse_transform(y_test_cls)
# y_pred_labels = le.inverse_transform(y_pred)

# # 예측 확률 (shape: [n_samples, n_classes])
# y_proba = log_best.predict_proba(X_test)

# # 클래스 이름 (원래 라벨로 변환)
# class_names = le.inverse_transform(np.arange(len(le.classes_)))

# # DataFrame으로 정리
# results_df = pd.DataFrame({
#     "y_true": y_true_labels,
#     "y_pred": y_pred_labels
# })

# # 확률 컬럼 추가 (각 클래스별 확률)
# proba_df = pd.DataFrame(y_proba, columns=[f"proba_{cls}" for cls in class_names])

# # 두 DataFrame 합치기
# results_df = pd.concat([results_df, proba_df], axis=1)

# # CSV 저장
# results_df.to_csv("logistic_test_predictions_with_proba.csv", index=False, encoding="utf-8-sig")
# print("💾 Saved test predictions with probabilities: logistic_test_predictions_with_proba.csv")

💾 Saved test predictions with probabilities: logistic_test_predictions_with_proba.csv


In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
import joblib

# ----- 타깃 준비 -----
y_train_reg = train_df["Bottleneck_val"].values
y_val_reg   = val_df["Bottleneck_val"].values
y_test_reg  = test_df["Bottleneck_val"].values

# ----- 학습/검증 합치기 -----
X_train_full = pd.concat([X_train, X_val], ignore_index=True)
y_train_full = np.concatenate([y_train_reg, y_val_reg])

X_train_full = X_train_full.astype(np.float32)
X_test = X_test.astype(np.float32)

# ----- 탐색 대상 파라미터 범위 (경량화) -----
param_dist = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 20, 40],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["sqrt"]
}

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# ----- RandomizedSearchCV (경량화) -----
search = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=5,              # ✅ 탐색 횟수 축소
    scoring="r2",
    cv=2,                  # ✅ 빠른 교차검증
    random_state=42,
    n_jobs=-1,
    verbose=1
)

print("🔍 RandomizedSearchCV 시작...")
search.fit(X_train_full, y_train_full)

print("✅ Best Params:", search.best_params_)
print("✅ Best CV R2:", search.best_score_)

# ----- 최적 모델로 테스트 -----
best_rf = search.best_estimator_
y_pred = best_rf.predict(X_test)

mae  = mean_absolute_error(y_test_reg, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred))
r2   = r2_score(y_test_reg, y_pred)
print(f"🎯 RandomForest Tuned | MAE={mae} | RMSE={rmse} | R2={r2}")

# ----- 모델 저장 -----
joblib.dump(best_rf, "rf_tuned_model.pkl")
print("💾 Saved: rf_tuned_model.pkl")

# ----- 예측 결과 저장 -----
results_df = pd.DataFrame({
    "y_true": y_test_reg,
    "y_pred": y_pred,
    "residual": y_test_reg - y_pred,
    "abs_error": np.abs(y_test_reg - y_pred),
    "pct_error": np.abs((y_test_reg - y_pred) / y_test_reg) * 100
})
results_df.to_csv("rf_test_predictions_tuned.csv", index=False, encoding="utf-8-sig")
print("💾 Saved: rf_test_predictions_tuned.csv")

🔍 RandomizedSearchCV 시작...
Fitting 2 folds for each of 5 candidates, totalling 10 fits
✅ Best Params: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 40}
✅ Best CV R2: 0.9930063040665469
🎯 RandomForest Tuned | MAE=1.535337446286351 | RMSE=4.321923875386631 | R2=0.9973829582438202
💾 Saved: rf_tuned_model.pkl
💾 Saved: rf_test_predictions_tuned.csv
